Алгоритм:

- строим решетку (grid) и соответствующее ей разбиение Вороного (vor4)
- Ищем все многогранники центрального региона и его диаметр
- составляем список граней центрального многогранника (list_faces)
- перебираю различные подрешетки для каждой ищу приведенный базис, нахожу длину минимального вектора и минимальное расстояние между векторами и их комбинациями:
- Ищем точку s (середина расстояния между точкой (0, 0, 0, 0) и центром ближайшего многогранника
- Нахожу расстояние от точки s до центрального многоранника
- Умножаю это расстояние на 2 и получаю расстояние между многранниками (центральным и ближайшим к нему многогранником из подрешетки)
- составляем словари, ключами которых являются координаты точек s, а значениями - минимальное хроматическое число (dict_det), максимальное запрещенное расстоянием (dict_dist), список матриц (dict_s), соответствующих данной точке s, хроматическому числу и запрещенному расстоянию
- составляем датафрейм (df), обьединяя все эти словари
- составляем список матриц для каждого подходящего хроматического числа (если запрещенное расстояние больше 1)

- central - список вершин центрального многогранника
- coords4 - список координат центров многогранников Вороного

- vor4.vertices - список координат вершин разбиенния Вороного
- vor4.regions - список индексов координат вершин многогранников разбиения Вороного (vor4.vertices)
- vor4.ridge_vertices - координаты вершин 3-мерных граней разбиенния Вороного через их индексы в vor4.vertices
- vor4.ridge_points - список пар центров многогранников Вороного между которыми есть ребро 2х многогранников через индексы в coords4 (по сути - соседние многогранники)

- edge_central_coords - список списков вершин каждой 3-мерной грани центрального многогранника
- edge_central - список индексов (в vor4.vertices)  координат каждой 3-мерной грани 
- list_faces - список 3х мерных граней центрального многогранника с разбивкой каждой 3х мерной грани на 2-мерные
- list_pairs - список центров многогранников Вороного, соседних с центральным (индексы в coords4)

- list_neigh_points - список координат центров многогранников, граничащих с центральным (по сути - вектора нормали к 3-мерным граням центрального многогранника)
- dict_point_edge - словарь: ключ - координаты соседней точки, значение - 3х мерная грань между соседней точкой и (0,0,0,0)
- index_point_edge - индекс грани в списке граней центрального многогранника (edge_central_coords)
- list_ridge_edge - список граней, в том же порядке, что и соседние точки (list_neigh_points) или что и соответствующие вектора нормали к данным граням
- v_norm - список нормированных векторов нормали (в том же порядке, что и изначальные точки(вектора) нормали)
- point_face_norm - точки, принадлежащие грани и лежащие на векторе нормали (середина вектора, соединяющего центры)

In [1]:
import numpy as np
from scipy.spatial import Voronoi
from scipy.spatial import distance
from sympy import symbols, Eq, solve
import math
import itertools
import copy
from scipy.spatial import Delaunay
import pandas as pd
import sys
from numpy import linalg as la
from itertools import *
from copy import deepcopy
from collections import defaultdict

from itertools import combinations
from math import prod
from sympy import factorint

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
grid = np.array([
    [2, 0, 0, 0],
    [1, 1, 0, 0],
    [1, 0, 1, 0],
    [1, 0, 0, 1]
], dtype = float)

grid

array([[2., 0., 0., 0.],
       [1., 1., 0., 0.],
       [1., 0., 1., 0.],
       [1., 0., 0., 1.]])

In [4]:
3+4

7

In [5]:
123

123

In [3]:
points_count = 8

coords4 = []
digits = range(- (points_count // 4), points_count - (points_count // 4))

for var in itertools.product(digits, repeat=4):
    coords4.append((grid.T).dot(var).tolist())
    
#coords4 # координаты центров

In [4]:
# строим диаграмму Вороного

vor4 = Voronoi(coords4)

In [5]:
# вершины диаграммы Вороного

vor4.vertices

array([[11. ,  5. , -2. ,  3. ],
       [-5. , -2. ,  2. , -2. ],
       [-3. , -2. , -2. , -2. ],
       ...,
       [16.5,  1.5,  3.5,  3.5],
       [17.5,  1.5,  3.5,  2.5],
       [17. ,  2. ,  5. ,  3. ]])

In [6]:
# регионы

vor4.regions[1]

[497,
 498,
 499,
 500,
 501,
 502,
 503,
 504,
 505,
 506,
 507,
 508,
 509,
 510,
 511,
 512,
 513,
 514,
 662,
 840,
 1067,
 1189,
 1214,
 1387]

In [7]:
# список конечных областей Вороного (-1 в координатах означает, что область бесконечна)

count = 0
vor4.regions_finite = []
for i in vor4.regions:
    if -1 not in i:
        vor4.regions_finite.append(i)
        count += 1
        
print(count)

1297


In [8]:
# максимальный размер региона (по числу вершин)

len_r_max = 0 # размер максимального региона
count_r = 0 # количество таких регионов

for i in range(len(vor4.regions)):
    len_r = len(vor4.regions[i])
    
    if len_r >= len_r_max:
        len_r_max = len_r
        count_r += 1 # считаем количество таких регионов
        
print('количество вершин =', len_r_max, 'количество =', count_r)

количество вершин = 24 количество = 1297


In [9]:
# находим суммарные расстояния от вершин многогранников до (0, 0, 0, 0) (sum_dist), затем находим минимальное 
# значение. Таким образом находим ближайший центральный многогранник

sum_dist_min = 1000

for i in range(1, len(vor4.regions)):
    len_r = len(vor4.regions[i])

    if -1 not in vor4.regions[i]: # регион должен быть конечным
        l = 0
        
        # для i региона перебираем все его вершины и для каждой ищем расстояние до [0.0, 0.0, 0.0, 0.0]
        for j in range(len_r):
            l += distance.euclidean(vor4.vertices[vor4.regions[i][j]], [0.0, 0.0, 0.0, 0.0])

        if l < sum_dist_min:
            sum_dist_min = l
            v_min = i

        
print ('суммарное расстояние =', sum_dist_min, 'индекс центрального региона =', v_min)

суммарное расстояние = 24.0 индекс центрального региона = 1320


In [10]:
# центральный регион

central = vor4.vertices[vor4.regions[v_min]]
central

array([[ 0.5, -0.5, -0.5, -0.5],
       [ 0.5, -0.5, -0.5,  0.5],
       [ 0.5, -0.5,  0.5,  0.5],
       [ 0. ,  0. ,  0. ,  1. ],
       [ 0.5, -0.5,  0.5, -0.5],
       [ 0. , -1. ,  0. ,  0. ],
       [-0.5, -0.5, -0.5,  0.5],
       [ 0. ,  0. ,  1. ,  0. ],
       [-0.5,  0.5,  0.5,  0.5],
       [-0.5,  0.5, -0.5,  0.5],
       [ 0. ,  0. , -1. ,  0. ],
       [-0.5, -0.5, -0.5, -0.5],
       [-0.5,  0.5, -0.5, -0.5],
       [-1. ,  0. ,  0. ,  0. ],
       [-0.5, -0.5,  0.5, -0.5],
       [-0.5,  0.5,  0.5, -0.5],
       [ 0. ,  0. ,  0. , -1. ],
       [-0.5, -0.5,  0.5,  0.5],
       [ 0.5,  0.5,  0.5,  0.5],
       [ 0.5,  0.5, -0.5,  0.5],
       [ 1. ,  0. ,  0. ,  0. ],
       [ 0.5,  0.5,  0.5, -0.5],
       [ 0.5,  0.5, -0.5, -0.5],
       [ 0. ,  1. ,  0. ,  0. ]])

In [11]:
# нахожу длину ребра центрального политопа

d_edge_min = 100
count = 0
for i in range(0, len(central)-1):
    for j in range(i + 1, len(central)):
        d_edge = distance.euclidean(central[i], central[j])
        if d_edge_min >= d_edge:
            d_edge_min = d_edge
            count += 1
            
d_edge_min, count
                           

(1.0, 96)

In [12]:
# индексы вершин центрального региона (координаты vor4.vertices)

vor4.regions[v_min]

[3194,
 3196,
 3922,
 3932,
 5504,
 6827,
 6828,
 7422,
 7423,
 7437,
 7441,
 7720,
 7721,
 7722,
 7729,
 7730,
 7733,
 7894,
 8159,
 8164,
 8165,
 8166,
 8167,
 8174]

In [13]:
# список 3х мерных граней 4х черных многогрнников через индексы вершин в vor4.vertices

vor4.ridge_vertices[0]

[499, 501, 502, 513, 514, 840]

In [14]:
# список вершин многогранников Вороного в координатах

vor4.vertices[499]

array([ 8.5,  4.5, -1.5, -1.5])

In [15]:
# номера координат центров многогранников Вороного (coords4) между которыми проходит грань, перпендикулярная вектору, 
# соединяющего эти центры 

vor4.ridge_points[1]

array([2953, 3016], dtype=int32)

In [16]:
# нашли индекс точки(центра) центрального многогранника

coords4.index([0.0, 0.0, 0.0, 0.0])

1170

In [17]:
coords4[1170]

[0.0, 0.0, 0.0, 0.0]

In [18]:
# нашли все центры многогранников у которых еть грань между многогранником и центральным многогранником
# индексы координат в coords4

list_pairs = []

for i in vor4.ridge_points:
    if 1170 in i:
        list_pairs.append(i)
        
list_pairs

[array([1169, 1170], dtype=int32),
 array([1162, 1170], dtype=int32),
 array([1617, 1170], dtype=int32),
 array([1610, 1170], dtype=int32),
 array([ 659, 1170], dtype=int32),
 array([1107, 1170], dtype=int32),
 array([ 666, 1170], dtype=int32),
 array([1114, 1170], dtype=int32),
 array([1106, 1170], dtype=int32),
 array([1234, 1170], dtype=int32),
 array([1171, 1170], dtype=int32),
 array([1178, 1170], dtype=int32),
 array([1170, 1618], dtype=int32),
 array([1170,  667], dtype=int32),
 array([1170, 1674], dtype=int32),
 array([1170, 1673], dtype=int32),
 array([1170, 1681], dtype=int32),
 array([1170, 1163], dtype=int32),
 array([1170, 1177], dtype=int32),
 array([1170,  723], dtype=int32),
 array([1170,  730], dtype=int32),
 array([1170,  722], dtype=int32),
 array([1170, 1226], dtype=int32),
 array([1170, 1233], dtype=int32)]

In [19]:
# составляем список центров многогранников, граничащих с центральным

list_neigh_points = [] # список центров многогранников, граничащих с центральным

for pair in list_pairs:
    if pair[0] == 1170:
        list_neigh_points.append(coords4[pair[1]])
    else:
        list_neigh_points.append(coords4[pair[0]])


list_neigh_points

[[-1.0, 0.0, 0.0, -1.0],
 [-1.0, 0.0, -1.0, 0.0],
 [0.0, -1.0, 0.0, -1.0],
 [0.0, -1.0, -1.0, 0.0],
 [-1.0, 0.0, 0.0, 1.0],
 [0.0, -1.0, 0.0, 1.0],
 [-1.0, 0.0, 1.0, 0.0],
 [0.0, -1.0, 1.0, 0.0],
 [-1.0, -1.0, 0.0, 0.0],
 [1.0, 1.0, 0.0, 0.0],
 [1.0, 0.0, 0.0, 1.0],
 [1.0, 0.0, 1.0, 0.0],
 [1.0, -1.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 1.0],
 [1.0, 0.0, -1.0, 0.0],
 [0.0, 0.0, -1.0, -1.0],
 [1.0, 0.0, 0.0, -1.0],
 [0.0, 0.0, -1.0, 1.0],
 [0.0, 0.0, 1.0, -1.0],
 [0.0, 1.0, 0.0, 1.0],
 [0.0, 1.0, 1.0, 0.0],
 [-1.0, 1.0, 0.0, 0.0],
 [0.0, 1.0, -1.0, 0.0],
 [0.0, 1.0, 0.0, -1.0]]

In [20]:
# трехмерные ячейки 

edge_central = [] # индексы координат вершин 3х мерных граней 


# перебирам 3х мерные грани 
for i in range(len(vor4.ridge_vertices)):
    count = 0

    
    # перебираем вершины каждой 3хмерной грани
    for j in range(len(vor4.ridge_vertices[i])):
        
        # если координата в списке координат центрального региона, увеличиваем count на 1
        if vor4.ridge_vertices[i][j] in vor4.regions[v_min]:
            count += 1 
            
    # если count = количеству вершин в многограннике, то мы нашли 
    if count == len(vor4.ridge_vertices[i]):
        edge_central.append(vor4.ridge_vertices[i])
        
# перевожу индексы вершин в координаты

edge_central_coords = [] # координаты вершин 3х мерных граней

for i in range(len(edge_central)):
    r = []
    for j in range(len(edge_central[i])):
        r.append(vor4.vertices[edge_central[i][j]])
    edge_central_coords.append(np.array(r))
    
edge_central_coords[0]

array([[-0.5, -0.5, -0.5, -0.5],
       [-0.5,  0.5, -0.5, -0.5],
       [-1. ,  0. ,  0. ,  0. ],
       [-0.5, -0.5,  0.5, -0.5],
       [-0.5,  0.5,  0.5, -0.5],
       [ 0. ,  0. ,  0. , -1. ]])

In [21]:
# ищем для каждого центра ближайшую трехмерную грань
# берем каждую точку из списка соседних центральных точек и ищем расстояние от нее до каждой 3х мерной грани
# суммируем и берем минимальное ссумарное расстояние. это и будет 3х мерная грань между точкой (0, 0, 0, 0)
# точкой из списка
# и вектор, соединяющий центры будет вектором нормали к данной 3х мерной грани


dict_point_edge = {} # словарь: ключ - соседняя точка, значение - 3х мерная грань между соседней точкой и (0,0,0,0)
index_point_edge = 0 # индекс грани в списке граней центрального многогранника (edge_central_coords[)
list_ridge_edge = [] # список граней, в том же порядке, что и соседние точки (list_neigh_points)


# перебираем все соседние центры

for point in range(len(list_neigh_points)):
    
    min_dist_point_edge = 1000

    # для каждой 3х мерной грани   
    for edge in range(len(edge_central_coords)):
        
        dist_point_edge = 0
        
        # для каждой вершины их 3х мерной грани считаем растояние до соседнего центра и суммируем
        for coord in edge_central_coords[edge]:
            
            dist_point_edge += distance.euclidean(coord, list_neigh_points[point])
            
        # если суммарное растояние меньше минимального, записываем новое суммарное растояние и индекс грани    
        if min_dist_point_edge > dist_point_edge:
            min_dist_point_edge = dist_point_edge
            index_point_edge = edge
            
    dict_point_edge[tuple(list_neigh_points[point])] = edge_central_coords[index_point_edge]
    list_ridge_edge.append(edge_central_coords[index_point_edge])
    
dict_point_edge

{(-1.0,
  0.0,
  0.0,
  -1.0): array([[-0.5, -0.5, -0.5, -0.5],
        [-0.5,  0.5, -0.5, -0.5],
        [-1. ,  0. ,  0. ,  0. ],
        [-0.5, -0.5,  0.5, -0.5],
        [-0.5,  0.5,  0.5, -0.5],
        [ 0. ,  0. ,  0. , -1. ]]),
 (-1.0,
  0.0,
  -1.0,
  0.0): array([[-0.5, -0.5, -0.5,  0.5],
        [-0.5,  0.5, -0.5,  0.5],
        [ 0. ,  0. , -1. ,  0. ],
        [-0.5, -0.5, -0.5, -0.5],
        [-0.5,  0.5, -0.5, -0.5],
        [-1. ,  0. ,  0. ,  0. ]]),
 (0.0,
  -1.0,
  0.0,
  -1.0): array([[ 0.5, -0.5, -0.5, -0.5],
        [ 0.5, -0.5,  0.5, -0.5],
        [ 0. , -1. ,  0. ,  0. ],
        [-0.5, -0.5, -0.5, -0.5],
        [-0.5, -0.5,  0.5, -0.5],
        [ 0. ,  0. ,  0. , -1. ]]),
 (0.0,
  -1.0,
  -1.0,
  0.0): array([[ 0.5, -0.5, -0.5, -0.5],
        [ 0.5, -0.5, -0.5,  0.5],
        [ 0. , -1. ,  0. ,  0. ],
        [-0.5, -0.5, -0.5,  0.5],
        [ 0. ,  0. , -1. ,  0. ],
        [-0.5, -0.5, -0.5, -0.5]]),
 (-1.0,
  0.0,
  0.0,
  1.0): array([[ 0. ,  0. ,  0. , 

In [22]:
list_neigh_points[10], list_ridge_edge[10]

([1.0, 0.0, 0.0, 1.0],
 array([[ 0.5, -0.5, -0.5,  0.5],
        [ 0.5, -0.5,  0.5,  0.5],
        [ 0. ,  0. ,  0. ,  1. ],
        [ 0.5,  0.5,  0.5,  0.5],
        [ 0.5,  0.5, -0.5,  0.5],
        [ 1. ,  0. ,  0. ,  0. ]]))

In [23]:
# проверка на ортогональность

for i in range(len(list_ridge_edge[10]) - 1):
    for j in range (1, len(list_ridge_edge[10])):
        print(np.array(list_neigh_points[10]).dot(np.array(list_ridge_edge[10][i] - list_ridge_edge[10][j])))
         

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [24]:
# список координат каждой 3х мерной грани (координаты даны индексами)

edge_central

[[7720, 7721, 7722, 7729, 7730, 7733],
 [6828, 7437, 7441, 7720, 7721, 7722],
 [3194, 5504, 6827, 7720, 7729, 7733],
 [3194, 3196, 6827, 6828, 7441, 7720],
 [3932, 6828, 7423, 7437, 7722, 7894],
 [3196, 3922, 3932, 6827, 6828, 7894],
 [7422, 7423, 7722, 7729, 7730, 7894],
 [3922, 5504, 6827, 7422, 7729, 7894],
 [6827, 6828, 7720, 7722, 7729, 7894],
 [8159, 8164, 8165, 8166, 8167, 8174],
 [3196, 3922, 3932, 8159, 8164, 8165],
 [3922, 5504, 7422, 8159, 8165, 8166],
 [3194, 3196, 3922, 5504, 6827, 8165],
 [3922, 3932, 7422, 7423, 7894, 8159],
 [3194, 3196, 7441, 8164, 8165, 8167],
 [3194, 7441, 7720, 7721, 7733, 8167],
 [3194, 5504, 7733, 8165, 8166, 8167],
 [3196, 3932, 6828, 7437, 7441, 8164],
 [5504, 7422, 7729, 7730, 7733, 8166],
 [3932, 7423, 7437, 8159, 8164, 8174],
 [7422, 7423, 7730, 8159, 8166, 8174],
 [7423, 7437, 7721, 7722, 7730, 8174],
 [7437, 7441, 7721, 8164, 8167, 8174],
 [7721, 7730, 7733, 8166, 8167, 8174]]

In [25]:
#длина списока координат каждой 3х мерной грани

len(edge_central_coords)

24

In [26]:
edge_central_coords[0]

array([[-0.5, -0.5, -0.5, -0.5],
       [-0.5,  0.5, -0.5, -0.5],
       [-1. ,  0. ,  0. ,  0. ],
       [-0.5, -0.5,  0.5, -0.5],
       [-0.5,  0.5,  0.5, -0.5],
       [ 0. ,  0. ,  0. , -1. ]])

In [27]:
# составляем список граней многогранника

list_faces = []

# берем каждый 3х мерный многогранник
for m in edge_central_coords:
    
    # составляем список ребер (расстояние между вершинами, соединенными ребрами = 1)
    list_edges = []
    for i in range(len(m)-1):
        for j in range(i+1, len(m)):
            k = []
            d = distance.euclidean(m[i], m[j])
            if d == 1:
                k.append(i)
                k.append(j)
                list_edges.append(k)
    # перебираем все комбинации ребер и ищем грани (если количество задействованных вершин в комбинации из 3х
    #  ребер = 3, то это грань)                       
    digits = list_edges
    list_faces_one = []  
    for var in itertools.combinations(digits, r=3):
        set_vet = set(var[0] + var[1] + var[2])
        set_vet_coord = []
        if len(set_vet) == 3:
            for i in range(len(set_vet)):
                set_vet_coord.append(m[list(set_vet)[i]])
            list_faces_one.append(set_vet_coord)
               
    list_faces.append(list_faces_one)
    
print('количество граней в одном зх мерном многограннике =', len(list_faces[1]))

количество граней в одном зх мерном многограннике = 8


In [28]:
# список трехмерных граней со списком 2х мерных граней

list_faces[0]

[[array([-0.5, -0.5, -0.5, -0.5]),
  array([-0.5,  0.5, -0.5, -0.5]),
  array([-1.,  0.,  0.,  0.])],
 [array([-0.5, -0.5, -0.5, -0.5]),
  array([-0.5,  0.5, -0.5, -0.5]),
  array([ 0.,  0.,  0., -1.])],
 [array([-0.5, -0.5, -0.5, -0.5]),
  array([-1.,  0.,  0.,  0.]),
  array([-0.5, -0.5,  0.5, -0.5])],
 [array([-0.5, -0.5, -0.5, -0.5]),
  array([-0.5, -0.5,  0.5, -0.5]),
  array([ 0.,  0.,  0., -1.])],
 [array([-0.5,  0.5, -0.5, -0.5]),
  array([-1.,  0.,  0.,  0.]),
  array([-0.5,  0.5,  0.5, -0.5])],
 [array([-0.5,  0.5, -0.5, -0.5]),
  array([-0.5,  0.5,  0.5, -0.5]),
  array([ 0.,  0.,  0., -1.])],
 [array([-1.,  0.,  0.,  0.]),
  array([-0.5, -0.5,  0.5, -0.5]),
  array([-0.5,  0.5,  0.5, -0.5])],
 [array([-0.5, -0.5,  0.5, -0.5]),
  array([-0.5,  0.5,  0.5, -0.5]),
  array([ 0.,  0.,  0., -1.])]]

In [29]:
# диаметр многогранника (ищем максимальное расстояние между вершинами) - 2

max_len = 2 * distance.euclidean(np.array([0, 0, 0, 0]), central[1])
max_len

2.0

In [30]:
# список вершин центрального многогранника
central

array([[ 0.5, -0.5, -0.5, -0.5],
       [ 0.5, -0.5, -0.5,  0.5],
       [ 0.5, -0.5,  0.5,  0.5],
       [ 0. ,  0. ,  0. ,  1. ],
       [ 0.5, -0.5,  0.5, -0.5],
       [ 0. , -1. ,  0. ,  0. ],
       [-0.5, -0.5, -0.5,  0.5],
       [ 0. ,  0. ,  1. ,  0. ],
       [-0.5,  0.5,  0.5,  0.5],
       [-0.5,  0.5, -0.5,  0.5],
       [ 0. ,  0. , -1. ,  0. ],
       [-0.5, -0.5, -0.5, -0.5],
       [-0.5,  0.5, -0.5, -0.5],
       [-1. ,  0. ,  0. ,  0. ],
       [-0.5, -0.5,  0.5, -0.5],
       [-0.5,  0.5,  0.5, -0.5],
       [ 0. ,  0. ,  0. , -1. ],
       [-0.5, -0.5,  0.5,  0.5],
       [ 0.5,  0.5,  0.5,  0.5],
       [ 0.5,  0.5, -0.5,  0.5],
       [ 1. ,  0. ,  0. ,  0. ],
       [ 0.5,  0.5,  0.5, -0.5],
       [ 0.5,  0.5, -0.5, -0.5],
       [ 0. ,  1. ,  0. ,  0. ]])

In [31]:
# нормируем вектора нормали
# ищем точки, принадлежащие граням и находящиеся на векторах нормали

v_norm = [] # здесь будем хранить нормированные вектора нормали
point_face_norm = [] #точки, принадлежащие граням и находящиеся на векторах нормали

for vec in list_neigh_points:
    v_norm.append(np.array(vec / np.linalg.norm(vec)))
    point_face_norm = np.array(vec) / 2
        
v_norm

[array([-0.70710678,  0.        ,  0.        , -0.70710678]),
 array([-0.70710678,  0.        , -0.70710678,  0.        ]),
 array([ 0.        , -0.70710678,  0.        , -0.70710678]),
 array([ 0.        , -0.70710678, -0.70710678,  0.        ]),
 array([-0.70710678,  0.        ,  0.        ,  0.70710678]),
 array([ 0.        , -0.70710678,  0.        ,  0.70710678]),
 array([-0.70710678,  0.        ,  0.70710678,  0.        ]),
 array([ 0.        , -0.70710678,  0.70710678,  0.        ]),
 array([-0.70710678, -0.70710678,  0.        ,  0.        ]),
 array([0.70710678, 0.70710678, 0.        , 0.        ]),
 array([0.70710678, 0.        , 0.        , 0.70710678]),
 array([0.70710678, 0.        , 0.70710678, 0.        ]),
 array([ 0.70710678, -0.70710678,  0.        ,  0.        ]),
 array([0.        , 0.        , 0.70710678, 0.70710678]),
 array([ 0.70710678,  0.        , -0.70710678,  0.        ]),
 array([ 0.        ,  0.        , -0.70710678, -0.70710678]),
 array([ 0.70710678,  0.

In [32]:
vertex_to_faces = [] # собой список списков 3D-граней для каждой вершины 
                        # (где индексы граней берутся из edge_central_coords)

for vertex in central:
    
    tamp_list = [] # список граней для текущей вершины
    
    for face in range(len(edge_central_coords)):

        if np.any(np.all(edge_central_coords[face] == vertex, axis=1)):
            tamp_list.append(face)
            
    vertex_to_faces.append(tamp_list)

vertex_to_faces

[[2, 3, 12, 14, 15, 16],
 [3, 5, 10, 12, 14, 17],
 [5, 7, 10, 11, 12, 13],
 [4, 5, 10, 13, 17, 19],
 [2, 7, 11, 12, 16, 18],
 [2, 3, 5, 7, 8, 12],
 [1, 3, 4, 5, 8, 17],
 [6, 7, 11, 13, 18, 20],
 [4, 6, 13, 19, 20, 21],
 [1, 4, 17, 19, 21, 22],
 [1, 3, 14, 15, 17, 22],
 [0, 1, 2, 3, 8, 15],
 [0, 1, 15, 21, 22, 23],
 [0, 1, 4, 6, 8, 21],
 [0, 2, 6, 7, 8, 18],
 [0, 6, 18, 20, 21, 23],
 [0, 2, 15, 16, 18, 23],
 [4, 5, 6, 7, 8, 13],
 [9, 10, 11, 13, 19, 20],
 [9, 10, 14, 17, 19, 22],
 [9, 10, 11, 12, 14, 16],
 [9, 11, 16, 18, 20, 23],
 [9, 14, 15, 16, 22, 23],
 [9, 19, 20, 21, 22, 23]]

In [33]:
# строим триангуляцию

delaunay = Delaunay(central)

In [34]:
class Edge2D:
    def __init__(self, vertex1, vertex2, face_center):
        '''
        Инициализация ребра.
        :vertex1: Координаты первой вершины ребра.
        :vertex2: Координаты второй вершины ребра.
        :face_center: Координаты центра грани.
        '''
        self.vertex1 = np.array(vertex1)
        self.vertex2 = np.array(vertex2)
        self.face_center = np.array(face_center)

        # Вычисляем середину ребра
        self.center = (self.vertex1 + self.vertex2) / 2

        # Вычисляем нормаль к ребру
        #edge_vector = self.vertex2 - self.vertex1
        self.normal = self.center - self.face_center  # Перпендикулярный вектор
        self.normal = self.normal / np.linalg.norm(self.normal)  # Нормализация
        
        self.bias = self.normal @ vertex1

        # Проверяем направление нормали (она должна быть направлена от центра грани)
        vector_to_center = self.face_center - self.center
        if np.dot(self.normal, vector_to_center) > 0:
            self.normal = -self.normal  # Меняем направление нормали

    def __repr__(self):
        return f"Edge2D(vertex1={self.vertex1}, vertex2={self.vertex2}, normal={self.normal}, \
                midpoint={self.center})"

In [35]:
class Face2D:
    def __init__(self, vertices, polyhedron_center):
        '''
        Инициализация 2D грани многогранника.
        '''
        self.vertices = vertices  # Координаты вершин грани
        self.parent_center = polyhedron_center # центр 3х мерной грани
        self.center = self._calculate_center()  # Центр грани
        self.normal = self._calculate_normal()  # Нормаль к грани
        self.edges = self._calculate_edges()  # ребра в 2х мерном многограннике
        self.bias = self.normal @ vertices[0]

    def _calculate_center(self):
        '''
        Вычисляет центр грани как среднее арифметическое координат вершин.
        '''
        return tuple(np.mean(self.vertices, axis=0))

    def _calculate_normal(self):

        # Вычисляем нормаль к грани как вектор из центра 3х мерного многогранника в центр 2х мерной грани
        normal_3d = np.array(self.center) - np.array(self.parent_center)

        # Нормализуем нормаль
        normal_3d = normal_3d / np.linalg.norm(normal_3d)

        return normal_3d

    def _calculate_edges(self):
        
        # составляем список ребер (расстояние между вершинами, соединенными ребрами = 1)
        list_edges = []
        
        for i in range(len(self.vertices)-1):
            for j in range(i+1, len(self.vertices)):
                
                d = distance.euclidean(self.vertices[i], self.vertices[j])
                
                if d == 1:
                    vertex1 = self.vertices[i]
                    vertex2 = self.vertices[j]  
                    edge = Edge2D(vertex1, vertex2, self.center)
                    list_edges.append(edge)
        
        return list_edges

    
    def __repr__(self):
        '''
        Возвращает строковое представление грани.
        '''
        return (f"Face2D(vertices={self.vertices}), center={self.center}, "
                f"normal={self.normal}")

In [36]:
class Polyhedron4D:
    def __init__(self, vertices, faces_list, normal):
        '''
        Инициализация многогранника в 4-мерном пространстве.
        :vertices: Список координат вершин многогранника .
        :normal: Вектор нормали к многограннику.
        '''
        self.vertices = vertices  # Координаты вершин
        self.normal = normal  # Нормаль к многограннику
        self.faces_list = faces_list  # Грани многогранника
        self.center = self._calculate_center()  # Центр многогранника
        self.faces = self._find_faces() # грани многогранника (как экземпляр класса)
        self.bias = normal @ vertices[0] # смещение

    def _calculate_center(self):
        '''
        Вычисляет центр многогранника как среднее арифметическое координат вершин.
        :return: Координаты центра.
        '''
        return tuple(np.mean(self.vertices, axis=0))

    
    def _find_faces(self):
        '''
        Находит грани многогранника и создаёт объекты Face2D.
        :return: Список объектов Face2D.
        '''
        
        faces = []

        for face_vertices in self.faces_list:
            face = Face2D(face_vertices, self.center)
            faces.append(face)

        return faces
    

    def __repr__(self):
        '''
        Возвращает строковое представление многогранника.
        '''
        return (f"Polyhedron4D(vertices={self.vertices}, normal={self.normal}, "
                f"center={self.center}, faces={self.faces})")


In [37]:
# расчитываем все грани центрального многогранника

polyhedrons = []  # Список для хранения всех объектов

for face_index in range(len(edge_central_coords)):
    polyhedron = Polyhedron4D(edge_central_coords[face_index], list_faces[face_index], v_norm[face_index])
    polyhedrons.append(polyhedron)
    

In [55]:
polyhedrons[0]

Polyhedron4D(vertices=[[-0.5 -0.5 -0.5 -0.5]
 [-0.5  0.5 -0.5 -0.5]
 [-1.   0.   0.   0. ]
 [-0.5 -0.5  0.5 -0.5]
 [-0.5  0.5  0.5 -0.5]
 [ 0.   0.   0.  -1. ]], normal=[-0.70710678  0.          0.         -0.70710678], center=(-0.5, 0.0, 0.0, -0.5), faces=[Face2D(vertices=[array([-0.5, -0.5, -0.5, -0.5]), array([-0.5,  0.5, -0.5, -0.5]), array([-1.,  0.,  0.,  0.])]), center=(-0.6666666666666666, 0.0, -0.3333333333333333, -0.3333333333333333), normal=[-0.40824829  0.         -0.81649658  0.40824829], Face2D(vertices=[array([-0.5, -0.5, -0.5, -0.5]), array([-0.5,  0.5, -0.5, -0.5]), array([ 0.,  0.,  0., -1.])]), center=(-0.3333333333333333, 0.0, -0.3333333333333333, -0.6666666666666666), normal=[ 0.40824829  0.         -0.81649658 -0.40824829], Face2D(vertices=[array([-0.5, -0.5, -0.5, -0.5]), array([-1.,  0.,  0.,  0.]), array([-0.5, -0.5,  0.5, -0.5])]), center=(-0.6666666666666666, -0.3333333333333333, 0.0, -0.3333333333333333), normal=[-0.40824829 -0.81649658  0.          0.408248

In [38]:
# через минимальное расстояние до вершины

def dist_to_s(polyhedrons, s):
    
    min_dist_to_pol = float('inf')
    index_proj = -1
    min_dist_vert_to_s = float('inf')
    min_vert = -1

    # находим ближайшую вершину к точке s
    
    for index in range(24):

        dist_vert_to_s = distance.euclidean(s, central[index])

        if dist_vert_to_s < min_dist_vert_to_s:
            min_dist_vert_to_s = dist_vert_to_s
            min_vert = index

    # находим расстояние и проекцию на центральный многогранник
    
    for i in vertex_to_faces[index]: # рассматриваем только грани,которым принадлежит ближайшая в точке s вершина

        d0 = polyhedrons[i].normal @ (s - polyhedrons[i].center)
        coord0 = s - (d0 + polyhedrons[i].bias)* polyhedrons[i].normal
        simplex = delaunay.find_simplex(coord0)
        
        if simplex != -1: # если проекция принадлежит центральному многораннику

            dist = abs(d0)
            coords_to_central = coord0

        else:
            for j in range(8):

                d1 = polyhedrons[i].faces[j].normal @ (coord0 - polyhedrons[i].faces[j].center)
                coord1 = coord0 - (d1 + polyhedrons[i].faces[j].bias)* polyhedrons[i].normal
                simplex = delaunay.find_simplex(coord0)

                if simplex != -1: # если проекция принадлежит центральному многораннику
                    dist = distance.euclidean(s, coord1, dtype = 'float')
                    coords_to_central = coord1

                else:
                    for k in range(3):
                        d2 = polyhedrons[i].faces[j].edges[k].normal @ (coord1 - \
                                                                 polyhedrons[i].faces[j].edges[k].center)
                        coord2 = coord1 - (d2 + polyhedrons[i].faces[j].edges[k].bias) * \
                                    polyhedrons[i].faces[j].edges[k].normal
                        simplex = delaunay.find_simplex(coord0)

                        if simplex != -1: # если проекция принадлежит центральному многораннику
                            dist = distance.euclidean(s, coord2, dtype = 'float')
                            coords_to_central = coord2

                        else:
                            d3 = distance.euclidean(s, polyhedrons[i].faces[j].edges[k].vertex1)
                            d4 = distance.euclidean(s, polyhedrons[i].faces[j].edges[k].vertex2)

                            if d3 < d4:
                                dist = d3
                                coords_to_central = polyhedrons[i].faces[j].edges[k].vertex1
                            else:
                                dist = d4
                                coords_to_central = polyhedrons[i].faces[j].edges[k].vertex2

        if min_dist_to_pol > dist:
            
            min_dist_to_pol = dist
            coords_proj = coords_to_central
            index_proj = i
        
        # если расстояние до какой-либо грани < 1, то дальше не считаем
        if  min_dist_to_pol < 1:
            
            return min_dist_to_pol, coords_proj, index_proj

        
    return min_dist_to_pol, coords_proj, index_proj

In [39]:
# через сумму растояний до вершин грани

def dist_to_s(polyhedrons, s):
    
    min_dist_to_pol = float('inf')
    index_proj = -1
    min_dist_vert_to_s = float('inf')
    
    # ищем ближайшую 3х мерную грань к точке s, находя сумму растояний до всех вершин грани

    min_sum_dist = float('inf') # минимальное суммарное растояние до вершин грани

    for index in range(len(list_ridge_edge)): # перебираем все 3-мерные грани
        sum_dist = 0

        for index_vert in range(6): # пребираем все вершины каждой 3-мерной грани
            sum_dist += distance.euclidean(s, np.array(list_ridge_edge[index][index_vert], dtype = 'float'))

        if sum_dist < min_sum_dist:
            min_sum_dist = sum_dist
            index_face = index


    # находим расстояние и проекцию на центральный многогранник

    i = index_face
    d0 = polyhedrons[i].normal @ (s - polyhedrons[i].center)
    coord0 = s - (d0 + polyhedrons[i].bias)* polyhedrons[i].normal
    simplex = delaunay.find_simplex(coord0)

    if simplex != -1: # если проекция принадлежит центральному многораннику

        dist = abs(d0)
        coords_to_central = coord0

    else:
        for j in range(8):

            d1 = polyhedrons[i].faces[j].normal @ (coord0 - polyhedrons[i].faces[j].center)
            coord1 = coord0 - (d1 + polyhedrons[i].faces[j].bias)* polyhedrons[i].normal
            simplex = delaunay.find_simplex(coord0)

            if simplex != -1: # если проекция принадлежит центральному многораннику
                dist = distance.euclidean(s, coord1, dtype = 'float')
                coords_to_central = coord1

            else:
                for k in range(3):
                    d2 = polyhedrons[i].faces[j].edges[k].normal @ (coord1 - \
                                                             polyhedrons[i].faces[j].edges[k].center)
                    coord2 = coord1 - (d2 + polyhedrons[i].faces[j].edges[k].bias) * \
                                polyhedrons[i].faces[j].edges[k].normal
                    simplex = delaunay.find_simplex(coord0)

                    if simplex != -1: # если проекция принадлежит центральному многораннику
                        dist = distance.euclidean(s, coord2, dtype = 'float')
                        coords_to_central = coord2

                    else:
                        d3 = distance.euclidean(s, polyhedrons[i].faces[j].edges[k].vertex1)
                        d4 = distance.euclidean(s, polyhedrons[i].faces[j].edges[k].vertex2)

                        if d3 < d4:
                            dist = d3
                            coords_to_central = polyhedrons[i].faces[j].edges[k].vertex1
                        else:
                            dist = d4
                            coords_to_central = polyhedrons[i].faces[j].edges[k].vertex2

    if min_dist_to_pol > dist:

        min_dist_to_pol = dist
        coords_proj = coords_to_central
        index_proj = i

    # если расстояние до какой-либо грани < 1, то дальше не считаем
    if  min_dist_to_pol < 1:

        return min_dist_to_pol, coords_proj, index_proj

        
    return min_dist_to_pol, coords_proj, index_proj

In [40]:
s = np.array([ 0.,  -1.,  -0.5,  0.5])
dist_to_s(polyhedrons, s)

(0.35355339059327373, array([ 0.  , -0.25,  0.25,  0.5 ]), 3)

In [41]:
polyhedrons[9].faces[0].edges[0].center

array([0.5, 0.5, 0. , 0.5])

In [42]:
min_d = 1.5

In [43]:
def gram_schmidt(v):

    w_1 = copy.deepcopy(v[0])
    w_n = copy.deepcopy(w_1)
    w_array = [deepcopy(w_n)]
    
    for n in range(1, len(v)):
        v_n = copy.deepcopy(v[n])
        w_n = copy.deepcopy(v_n)

        for j in range(n):
            w_j = deepcopy(w_array[j])
            if not any(w_j):
                continue
            w_n -= np.dot(v_n, w_j) / np.dot(w_j, w_j) * w_j
        
        w_array += [w_n]
          
    return w_array

In [44]:
d = 0.75

def mu(b_i, b_j):
    return np.dot(b_i, b_j) / np.dot(b_j, b_j) if np.dot(b_j, b_j) != 0 else 0


def LLL(l_basis):

    ortho = gram_schmidt(l_basis)

    k = 1
    n = len(ortho)

    while k < n:
        for j in range(k - 1, -1, -1):
            proj = mu(l_basis[k], ortho[j])
            if abs(proj) > 1/2:
                l_basis[k] -= l_basis[j] * round(proj)
                ortho = copy.deepcopy(gram_schmidt(l_basis))
            #print(l_basis, 111)

        if np.dot(ortho[k], ortho[k]) >= (d - (mu(l_basis[k], ortho[k-1]))**2) * (np.dot(ortho[k-1], ortho[k-1])):
            #print(l_basis, 222)
            k += 1

        else:
            s = copy.deepcopy(l_basis[k-1])
            l_basis[k-1] = copy.deepcopy(l_basis[k])
            l_basis[k] = copy.deepcopy(s)
            ortho = gram_schmidt(l_basis)
            k = max(k-1, 1)
            #print(l_basis, 333)

    return l_basis

In [45]:
# генерация комбинаций

def generate_factor_combinations(factors):
    if not factors:
        return [[]]
    
    first, *rest = factors
    rest_combinations = generate_factor_combinations(rest)
    
    result = []
    for comb in rest_combinations:
        result.append([first] + comb)
        result.append([first * (comb[0] if comb else 1)] + (comb[1:] if len(comb) > 1 else []))
    
    return result

In [46]:
# находит все возможные разложения числа на простые множители

def compute_factorizations(n):
    
    # находим простые множители числа
    
    factors = [] # список простых множителей
    
    for prime, exp in factorint(n).items():
        factors.extend([prime] * exp)
    
    combinations = []
    combinations = generate_factor_combinations(factors) # генерируем список всех возможных комбинаций
    
    unique = set(tuple(sorted(comb)) for comb in combinations) # убираем дубликаты и сортируем
    unique_combinations = [list(comb) for comb in sorted(unique)]
    
    return unique_combinations
    


In [50]:
list_all_factorizations = compute_factorizations(54)
list_all_factorizations

[[2, 3, 3, 3], [2, 3, 9], [2, 27], [3, 3, 6], [3, 18], [6, 9], [54]]

In [51]:
factorint(54)

{2: 1, 3: 3}

In [48]:
def pad_lists_with_ones(list_of_lists):
    # Проходим по каждому списку в основном списке
    for i in range(len(list_of_lists)):
        # Если длина списка меньше 4
        if len(list_of_lists[i]) < 4:
            # Добавляем в начало столько единиц, сколько не хватает до 4
            list_of_lists[i] = [1] * (4 - len(list_of_lists[i])) + list_of_lists[i]
    return list_of_lists


In [49]:
# вариант 1 - считаем s - считаем как середина самого короткого вектора lll-базиса

list_mat = []
list_lll = []
list_s1 = []
list_dist = []


# задаю величину определителя

det_mat = 54
print(det_mat)

list_all_factorizations = compute_factorizations(det_mat)
list_diag_el = pad_lists_with_ones(list_all_factorizations)

for diag_el in list_diag_el:
    mat = np.array([[diag_el[0], 0, 0, 0], 
                    [0, diag_el[1], 0, 0], 
                    [0, 0, diag_el[2], 0], 
                    [0, 0, 0, diag_el[3]]], float)
    max_num_col3 = diag_el[3]
    max_num_col2 = diag_el[2]
    max_num_col1 = diag_el[1]
    #max_num_col0 = diag_el[0]
    print(max_num_col1, max_num_col2, max_num_col3)

    for i1 in range(max_num_col3):
        mat[2][3] = i1
        print(i1)
        for i2 in range(max_num_col3):
            mat[1][3] = i2
            for i3 in range(max_num_col2):
                mat[1][2] = i3
                for i4 in range(max_num_col3):
                    mat[0][3] = i4
                    for i5 in range(max_num_col2):
                        mat[0][2] = i5
                        for i6 in range(max_num_col1):
                            mat[0][1] = i6
                            
                            sub_grid = np.dot(mat, grid)
                            basis = LLL(sub_grid)

                            # ищем самый короткий вектор
                            min_dist = 100
                            index = 0
                            for i in range(4):
                                d_vec = np.linalg.norm(basis[i])
                                if d_vec < min_dist:
                                    min_dist = d_vec
                                    index = i

                            # определяем точку s как половину длины самого короткого базисного вектора
                            if min_dist > min_d:
                                s1 = basis[index] * 0.5
                                #print(mat, s1)
                                dist_s1, coords_proj_s, index_pol = dist_to_s(polyhedrons, s1)
                                
                                if dist_s1 > 1:
                                    print(dist_s1, s1, mat)
                                    list_dist.append(dist_s1)
                                    list_mat.append(mat)
                                    list_lll.append(basis)
                                    list_s1.append(s1)

54
3 3 3
0
1
2
2 3 9
0
1.0606601717798212 [1.5 1.  0.  0.5] [[1. 1. 0. 6.]
 [0. 2. 0. 1.]
 [0. 0. 3. 0.]
 [0. 0. 0. 9.]]
1
2
3
4
5
6
7
8
1 2 27
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
1.0606601717798212 [ 1.5 -0.5  1.   0. ] [[ 1.  0.  1.  9.]
 [ 0.  1.  1. 24.]
 [ 0.  0.  2. 15.]
 [ 0.  0.  0. 27.]]
16
17
18
19
20
21
22
23
24
25
26
3 3 6
0
1
2
3
4
5
1.0606601717798212 [-1.  -1.5  0.   0.5] [[1. 2. 2. 0.]
 [0. 3. 0. 5.]
 [0. 0. 3. 5.]
 [0. 0. 0. 6.]]
1 3 18
0
1.0606601717798212 [1.5 0.5 0.  1. ] [[ 1.  0.  2. 12.]
 [ 0.  1.  0.  2.]
 [ 0.  0.  3.  0.]
 [ 0.  0.  0. 18.]]
1.0606601717798212 [1.  0.  0.5 1.5] [[ 1.  0.  2. 15.]
 [ 0.  1.  2.  4.]
 [ 0.  0.  3.  0.]
 [ 0.  0.  0. 18.]]
1.0606601717798212 [-1.5  0.5  0.  -1. ] [[ 1.  0.  2. 12.]
 [ 0.  1.  2. 10.]
 [ 0.  0.  3.  0.]
 [ 0.  0.  0. 18.]]
1.0606601717798212 [1.  0.  0.5 1.5] [[ 1.  0.  2. 15.]
 [ 0.  1.  0. 11.]
 [ 0.  0.  3.  0.]
 [ 0.  0.  0. 18.]]
1
2
3
4
5
6
7
8
9
10
11
12
1.0606601717798212 [ 0.5 -1.5  0.   1. ] [[ 1.  0. 

In [ ]:
list_mat = []
list_lll = []
list_s1 = []
list_dist = []


# задаю величину определителя

det_mat = 44
print(det_mat)


list_all_factorizations = compute_factorizations(det_mat)
list_diag_el = pad_lists_with_ones(list_all_factorizations)

for diag_el in list_diag_el:
    mat = np.array([[diag_el[0], 0, 0, 0], 
                    [0, diag_el[1], 0, 0], 
                    [0, 0, diag_el[2], 0], 
                    [0, 0, 0, diag_el[3]]], float)
    max_num_col3 = diag_el[3]
    max_num_col2 = diag_el[2]
    max_num_col1 = diag_el[1]
    #max_num_col0 = diag_el[0]
    print(max_num_col1, max_num_col2, max_num_col3)

    for i1 in range(max_num_col3):
        mat[2][3] = i1
        print(i1)
        for i2 in range(max_num_col3):
            mat[1][3] = i2
            for i3 in range(max_num_col2):
                mat[1][2] = i3
                for i4 in range(max_num_col3):
                    mat[0][3] = i4
                    for i5 in range(max_num_col2):
                        mat[0][2] = i5
                        for i6 in range(max_num_col1):
                            mat[0][1] = i6
                            
                            sub_grid = np.dot(mat, grid)
                            basis = LLL(sub_grid)

                            # ищем самый короткий вектор
                            min_dist = 100
                            index = 0
                            for i in range(4):
                                d_vec = np.linalg.norm(basis[i])
                                if d_vec < min_dist:
                                    min_dist = d_vec
                                    index = i

                            # определяем точку s как половину длины самого короткого базисного вектора
                            if min_dist > min_d:
                                s1 = basis[index] * 0.5
                                #print(mat, s1)
                                dist_s1, coords_proj_s, index_pol = dist_to_s(polyhedrons, s1)
                                
                                if dist_s1 > 1:
                                    print(dist_s1, s1, mat)
                                    list_dist.append(dist_s1)
                                    list_mat.append(mat)
                                    list_lll.append(basis)
                                    list_s1.append(s1)

In [ ]:
import time

In [ ]:
# с проверкой, есть ли точка в списке и с переводом s в положительные координаты

start = time.time()

list_mat = []
list_lll = []
list_s1 = []
list_dist = []


# задаю величину определителя
det_mat = 54
print(det_mat)


list_all_factorizations = compute_factorizations(det_mat)
list_diag_el = pad_lists_with_ones(list_all_factorizations)

for diag_el in list_diag_el:
    mat = np.array([[diag_el[0], 0, 0, 0], 
                    [0, diag_el[1], 0, 0], 
                    [0, 0, diag_el[2], 0], 
                    [0, 0, 0, diag_el[3]]], float)
    max_num_col3 = diag_el[3]
    max_num_col2 = diag_el[2]
    max_num_col1 = diag_el[1]
    
    print(max_num_col1, max_num_col2, max_num_col3)

    for i1 in range(max_num_col3):
        mat[2][3] = i1
        print(i1)
        for i2 in range(max_num_col3):
            mat[1][3] = i2
            for i3 in range(max_num_col2):
                mat[1][2] = i3
                for i4 in range(max_num_col3):
                    mat[0][3] = i4
                    for i5 in range(max_num_col2):
                        mat[0][2] = i5
                        for i6 in range(max_num_col1):
                            mat[0][1] = i6
                            
                            sub_grid = np.dot(mat, grid)
                            basis = LLL(sub_grid)

                            # ищем самый короткий вектор
                            min_dist = 100
                            index = 0
                            for i in range(4):
                                d_vec = np.linalg.norm(basis[i])
                                if d_vec < min_dist:
                                    min_dist = d_vec
                                    index = i

                            # определяем точку s как половину длины самого короткого базисного вектора
                            if min_dist > min_d:
                                s1 = basis[index] * 0.5
                                
                                s1_positive = np.abs(s1) # делаем все координаты положительными 
                                
                                # проверяем есть ли данная точка уже в списке проверенных точек
                                if not any(np.array_equal(s1_positive, arr) for arr in list_s1):
                                    dist_s1, coords_proj_s, index_pol = dist_to_s(polyhedrons, s1_positive)

                                    if dist_s1 > 1:
                                        print(dist_s1, s1_positive, mat)
                                        list_dist.append(dist_s1)
                                        list_mat.append(mat)
                                        list_lll.append(basis)
                                        list_s1.append(s1_positive)
                                        
end = time.time()
start - end

In [ ]:
# с проверкой, есть ли точка в списке и с переводом s в положительные координаты

start = time.time()

list_mat = []
list_lll = []
list_s1 = []
list_dist = []


# задаю величину определителя
det_mat = 50
print(det_mat)


list_all_factorizations = compute_factorizations(det_mat)
list_diag_el = pad_lists_with_ones(list_all_factorizations)

for diag_el in list_diag_el:
    mat = np.array([[diag_el[0], 0, 0, 0], 
                    [0, diag_el[1], 0, 0], 
                    [0, 0, diag_el[2], 0], 
                    [0, 0, 0, diag_el[3]]], float)
    max_num_col3 = diag_el[3]
    max_num_col2 = diag_el[2]
    max_num_col1 = diag_el[1]
    
    print(max_num_col1, max_num_col2, max_num_col3)

    for i1 in range(max_num_col3):
        mat[2][3] = i1
        print(i1)
        for i2 in range(max_num_col3):
            mat[1][3] = i2
            for i3 in range(max_num_col2):
                mat[1][2] = i3
                for i4 in range(max_num_col3):
                    mat[0][3] = i4
                    for i5 in range(max_num_col2):
                        mat[0][2] = i5
                        for i6 in range(max_num_col1):
                            mat[0][1] = i6
                            
                            sub_grid = np.dot(mat, grid)
                            basis = LLL(sub_grid)

                            # ищем самый короткий вектор
                            min_dist = 100
                            index = 0
                            for i in range(4):
                                d_vec = np.linalg.norm(basis[i])
                                if d_vec < min_dist:
                                    min_dist = d_vec
                                    index = i

                            # определяем точку s как половину длины самого короткого базисного вектора
                            if min_dist > min_d:
                                s1 = basis[index] * 0.5
                                
                                s1_positive = np.abs(s1) # делаем все координаты положительными 
                                
                                # проверяем есть ли данная точка уже в списке проверенных точек
                                if not any(np.array_equal(s1_positive, arr) for arr in list_s1):
                                    dist_s1, coords_proj_s, index_pol = dist_to_s(polyhedrons, s1_positive)

                                    if dist_s1 > 1:
                                        print(dist_s1, s1_positive, mat)
                                        list_dist.append(dist_s1)
                                        list_mat.append(mat)
                                        list_lll.append(basis)
                                        list_s1.append(s1_positive)
                                        
end = time.time()
start - end

In [ ]:
# с проверкой, есть ли точка в списке и с переводом s в положительные координаты

start = time.time()

list_mat = []
list_lll = []
list_s1 = []
list_dist = []


# задаю величину определителя
det_mat = 52
print(det_mat)


list_all_factorizations = compute_factorizations(det_mat)
list_diag_el = pad_lists_with_ones(list_all_factorizations)

for diag_el in list_diag_el:
    mat = np.array([[diag_el[0], 0, 0, 0], 
                    [0, diag_el[1], 0, 0], 
                    [0, 0, diag_el[2], 0], 
                    [0, 0, 0, diag_el[3]]], float)
    max_num_col3 = diag_el[3]
    max_num_col2 = diag_el[2]
    max_num_col1 = diag_el[1]
    
    print(max_num_col1, max_num_col2, max_num_col3)

    for i1 in range(max_num_col3):
        mat[2][3] = i1
        print(i1)
        for i2 in range(max_num_col3):
            mat[1][3] = i2
            for i3 in range(max_num_col2):
                mat[1][2] = i3
                for i4 in range(max_num_col3):
                    mat[0][3] = i4
                    for i5 in range(max_num_col2):
                        mat[0][2] = i5
                        for i6 in range(max_num_col1):
                            mat[0][1] = i6
                            
                            sub_grid = np.dot(mat, grid)
                            basis = LLL(sub_grid)

                            # ищем самый короткий вектор
                            min_dist = 100
                            index = 0
                            for i in range(4):
                                d_vec = np.linalg.norm(basis[i])
                                if d_vec < min_dist:
                                    min_dist = d_vec
                                    index = i

                            # определяем точку s как половину длины самого короткого базисного вектора
                            if min_dist > min_d:
                                s1 = basis[index] * 0.5
                                
                                s1_positive = np.abs(s1) # делаем все координаты положительными 
                                
                                # проверяем есть ли данная точка уже в списке проверенных точек
                                if not any(np.array_equal(s1_positive, arr) for arr in list_s1):
                                    dist_s1, coords_proj_s, index_pol = dist_to_s(polyhedrons, s1_positive)

                                    if dist_s1 > 1:
                                        print(dist_s1, s1_positive, mat)
                                        list_dist.append(dist_s1)
                                        list_mat.append(mat)
                                        list_lll.append(basis)
                                        list_s1.append(s1_positive)
                                        
end = time.time()
-start + end

In [ ]:
# с проверкой, есть ли точка в списке и с переводом s в положительные координаты

start = time.time()

list_mat = []
list_lll = []
list_s1 = []
list_dist = []


# задаю величину определителя
det_mat = 49
print(det_mat)


list_all_factorizations = compute_factorizations(det_mat)
list_diag_el = pad_lists_with_ones(list_all_factorizations)

for diag_el in list_diag_el:
    mat = np.array([[diag_el[0], 0, 0, 0], 
                    [0, diag_el[1], 0, 0], 
                    [0, 0, diag_el[2], 0], 
                    [0, 0, 0, diag_el[3]]], float)
    max_num_col3 = diag_el[3]
    max_num_col2 = diag_el[2]
    max_num_col1 = diag_el[1]
    
    print(max_num_col1, max_num_col2, max_num_col3)

    for i1 in range(max_num_col3):
        mat[2][3] = i1
        print(i1)
        for i2 in range(max_num_col3):
            mat[1][3] = i2
            for i3 in range(max_num_col2):
                mat[1][2] = i3
                for i4 in range(max_num_col3):
                    mat[0][3] = i4
                    for i5 in range(max_num_col2):
                        mat[0][2] = i5
                        for i6 in range(max_num_col1):
                            mat[0][1] = i6
                            
                            sub_grid = np.dot(mat, grid)
                            basis = LLL(sub_grid)

                            # ищем самый короткий вектор
                            min_dist = 100
                            index = 0
                            for i in range(4):
                                d_vec = np.linalg.norm(basis[i])
                                if d_vec < min_dist:
                                    min_dist = d_vec
                                    index = i

                            # определяем точку s как половину длины самого короткого базисного вектора
                            if min_dist > min_d:
                                s1 = basis[index] * 0.5
                                
                                s1_positive = np.abs(s1) # делаем все координаты положительными 
                                
                                # проверяем есть ли данная точка уже в списке проверенных точек
                                if not any(np.array_equal(s1_positive, arr) for arr in list_s1):
                                    dist_s1, coords_proj_s, index_pol = dist_to_s(polyhedrons, s1_positive)

                                    if dist_s1 > 1:
                                        print(dist_s1, s1_positive, mat)
                                        list_dist.append(dist_s1)
                                        list_mat.append(mat)
                                        list_lll.append(basis)
                                        list_s1.append(s1_positive)
                                        
end = time.time()
start - end

In [ ]:
# с проверкой, есть ли точка в списке и с переводом s в положительные координаты

start = time.time()

list_mat = []
list_lll = []
list_s1 = []
list_dist = []


# задаю величину определителя
det_mat = 40
print(det_mat)


list_all_factorizations = compute_factorizations(det_mat)
list_diag_el = pad_lists_with_ones(list_all_factorizations)

for diag_el in list_diag_el:
    mat = np.array([[diag_el[0], 0, 0, 0], 
                    [0, diag_el[1], 0, 0], 
                    [0, 0, diag_el[2], 0], 
                    [0, 0, 0, diag_el[3]]], float)
    max_num_col3 = diag_el[3]
    max_num_col2 = diag_el[2]
    max_num_col1 = diag_el[1]
    
    print(max_num_col1, max_num_col2, max_num_col3)

    for i1 in range(max_num_col3):
        mat[2][3] = i1
        print(i1)
        for i2 in range(max_num_col3):
            mat[1][3] = i2
            for i3 in range(max_num_col2):
                mat[1][2] = i3
                for i4 in range(max_num_col3):
                    mat[0][3] = i4
                    for i5 in range(max_num_col2):
                        mat[0][2] = i5
                        for i6 in range(max_num_col1):
                            mat[0][1] = i6
                            
                            sub_grid = np.dot(mat, grid)
                            basis = LLL(sub_grid)

                            # ищем самый короткий вектор
                            min_dist = 100
                            index = 0
                            for i in range(4):
                                d_vec = np.linalg.norm(basis[i])
                                if d_vec < min_dist:
                                    min_dist = d_vec
                                    index = i

                            # определяем точку s как половину длины самого короткого базисного вектора
                            if min_dist > min_d:
                                s1 = basis[index] * 0.5
                                
                                s1_positive = np.abs(s1) # делаем все координаты положительными 
                                
                                # проверяем есть ли данная точка уже в списке проверенных точек
                                if not any(np.array_equal(s1_positive, arr) for arr in list_s1):
                                    dist_s1, coords_proj_s, index_pol = dist_to_s(polyhedrons, s1_positive)

                                    if dist_s1 > 1:
                                        print(dist_s1, s1_positive, mat)
                                        list_dist.append(dist_s1)
                                        list_mat.append(mat)
                                        list_lll.append(basis)
                                        list_s1.append(s1_positive)
                                        
end = time.time()
start - end